In [2]:
from customtkinter import *
from tkinter import messagebox
from datetime import datetime
import heapq

# event class definition
class Event:
    def __init__(self, event_name, event_date, priority, is_all_day=False, schedules=None):
        self.event_name = event_name
        self.event_date = datetime.strptime(event_date, "%Y-%m-%d").date()  # parse date
        self.priority = priority
        self.is_all_day = is_all_day
        self.schedules = schedules if schedules else []  # schedule list

    def __lt__(self, other):
        if self.event_date == other.event_date:
            return self.priority_value() > other.priority_value()  # compare priority
        return self.event_date < other.event_date  # compare date

    def priority_value(self):
        return {"High": 3, "Medium": 2, "Low": 1}.get(self.priority, 0)  # priority mapping

    def __str__(self):
        schedules_str = ", ".join([f"{start} to {end}" for start, end in self.schedules])
        all_day_str = " | All Day" if self.is_all_day else ""  # check all-day
        return f"{self.event_name} | {self.event_date} | Priority: {self.priority}{all_day_str} | Schedules: {schedules_str or 'None'}"


class EventManagerApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Event Manager")  # set title
        self.root.geometry("900x600")  # set window size
        self.events = []  # event list

        set_default_color_theme("blue")  # set color theme

        # create widgets
        self.create_widgets()

    def create_widgets(self):
        # sidebar frame
        self.sidebar_frame = CTkFrame(self.root, width=200, corner_radius=10, fg_color="#D1D1D6")
        self.sidebar_frame.grid(row=0, column=0, rowspan=2, sticky="ns", padx=20, pady=20)

        # create event button
        self.create_event_button = CTkButton(self.sidebar_frame, text="Create Event", command=self.show_create_event, width=180)
        self.create_event_button.grid(row=0, column=0, padx=20, pady=20)

        # show events button
        self.show_event_button = CTkButton(self.sidebar_frame, text="Show Events", command=self.show_show_events, width=180)
        self.show_event_button.grid(row=1, column=0, padx=20, pady=20)

        # content frame
        self.content_frame = CTkFrame(self.root)
        self.content_frame.grid(row=0, column=1, padx=20, pady=20, sticky="nsew")

        # default view
        self.show_create_event()

    def show_create_event(self):
        # clear content frame
        for widget in self.content_frame.winfo_children():
            widget.destroy()

        # event name entry
        self.event_name_entry = CTkEntry(self.content_frame, placeholder_text="Event Name", width=300)
        self.event_name_entry.grid(row=0, column=0, pady=10)

        # event date entry
        self.event_date_entry = CTkEntry(self.content_frame, placeholder_text="Event Date (YYYY-MM-DD)", width=300)
        self.event_date_entry.grid(row=1, column=0, pady=10)

        # priority combobox
        self.priority_combobox = CTkComboBox(self.content_frame, values=["High", "Medium", "Low"], width=300)
        self.priority_combobox.set("Medium")
        self.priority_combobox.grid(row=2, column=0, pady=10)

        # all-day switch
        self.is_all_day_var = BooleanVar()
        self.all_day_checkbox = CTkSwitch(self.content_frame, text="All Day Event", variable=self.is_all_day_var, command=self.toggle_schedule_fields)
        self.all_day_checkbox.grid(row=3, column=0, pady=10)

        # time entry fields
        self.start_time_entry = CTkEntry(self.content_frame, placeholder_text="Start Time (HH:MM)", width=150)
        self.start_time_entry.grid(row=4, column=0, pady=5)

        self.end_time_entry = CTkEntry(self.content_frame, placeholder_text="End Time (HH:MM)", width=150)
        self.end_time_entry.grid(row=4, column=1, pady=5)

        # add event button
        self.add_event_button = CTkButton(self.content_frame, text="Add Event", command=self.add_event)
        self.add_event_button.grid(row=5, column=0, columnspan=2, pady=10)

    def show_show_events(self):
        # clear content frame
        for widget in self.content_frame.winfo_children():
            widget.destroy()

        # sorting buttons
        self.sort_by_priority_button = CTkButton(self.content_frame, text="Sort by Priority", command=self.sort_by_priority)
        self.sort_by_priority_button.grid(row=0, column=0, pady=10)

        self.sort_by_date_button = CTkButton(self.content_frame, text="Sort by Date", command=self.sort_by_date)
        self.sort_by_date_button.grid(row=0, column=1, pady=10)

        # remove low-priority button
        self.remove_non_top_button = CTkButton(self.content_frame, text="Remove Non-Top Events", command=self.remove_non_top_events)
        self.remove_non_top_button.grid(row=1, column=0, pady=10)

        # date filter entries
        self.start_date_filter = CTkEntry(self.content_frame, placeholder_text="Start Date (YYYY-MM-DD)", width=150)
        self.start_date_filter.grid(row=2, column=0, pady=5)

        self.end_date_filter = CTkEntry(self.content_frame, placeholder_text="End Date (YYYY-MM-DD)", width=150)
        self.end_date_filter.grid(row=2, column=1, pady=5)

        self.filter_button = CTkButton(self.content_frame, text="Show Events in Date Range", command=self.filter_events_by_date)
        self.filter_button.grid(row=2, column=2, pady=10)

        # events listbox
        self.events_listbox = CTkTextbox(self.content_frame, width=500, height=200)
        self.events_listbox.grid(row=3, column=0, columnspan=3, pady=10)

        # update events list
        self.update_event_listbox()

    def toggle_schedule_fields(self):
        if self.is_all_day_var.get():
            self.start_time_entry.configure(state="disabled")  # disable time fields
            self.end_time_entry.configure(state="disabled")
        else:
            self.start_time_entry.configure(state="normal")  # enable time fields
            self.end_time_entry.configure(state="normal")

    def add_event(self):
        event_name = self.event_name_entry.get().strip()
        event_date = self.event_date_entry.get().strip()
        priority = self.priority_combobox.get().strip()
        is_all_day = self.is_all_day_var.get()
        start_time = self.start_time_entry.get().strip() if not is_all_day else None
        end_time = self.end_time_entry.get().strip() if not is_all_day else None

        if not event_name or not event_date or not priority:
            messagebox.showerror("Input Error", "Please fill in all required fields!")
            return
        try:
            # validate date and time
            datetime.strptime(event_date, "%Y-%m-%d")
            if start_time and end_time:
                datetime.strptime(start_time, "%H:%M")
                datetime.strptime(end_time, "%H:%M")
        except ValueError:
            messagebox.showerror("Input Error", "Invalid date or time format!")
            return

        schedules = [(start_time, end_time)] if start_time and end_time else []
        event = Event(event_name, event_date, priority, is_all_day, schedules)
        heapq.heappush(self.events, event)  # add event
        self.update_event_listbox()

    def remove_non_top_events(self):
        # remove low priority events
        self.events = [e for e in self.events if e.priority != "Low"]
        heapq.heapify(self.events)  # re-sort events
        self.update_event_listbox()

    def sort_by_priority(self):
        # sort events by priority
        self.events.sort(key=lambda e: e.priority_value(), reverse=True)
        self.update_event_listbox()

    def sort_by_date(self):
        # sort events by date
        self.events.sort(key=lambda e: e.event_date, reverse=False)
        self.update_event_listbox()

    def filter_events_by_date(self):
        # filter events by date range
        start_date = self.start_date_filter.get().strip()
        end_date = self.end_date_filter.get().strip()
        try:
            start = datetime.strptime(start_date, "%Y-%m-%d").date()
            end = datetime.strptime(end_date, "%Y-%m-%d").date()
        except ValueError:
            messagebox.showerror("Input Error", "Invalid date format!")
            return
        filtered_events = [e for e in self.events if start <= e.event_date <= end]
        self.update_event_listbox(filtered_events)

    def update_event_listbox(self, events=None):
        # update listbox display
        self.events_listbox.delete(1.0, END)
        self.events_listbox.insert(END, "Name | Date | Priority | Time\n")
        for event in events or self.events:
            event_date = event.event_date.strftime("%Y-%m-%d")  # format date
            self.events_listbox.insert(END, f"{event.event_name} | {event_date} | {event.priority} | {', '.join([f'{start} to {end}' for start, end in event.schedules]) or 'None'}\n")


# --- Main ---
if __name__ == "__main__":
    app = CTk()  # create app
    app.geometry("900x600")  # set size
    event_manager_app = EventManagerApp(app)  # run app
    app.mainloop()  # start event loop


ModuleNotFoundError: No module named 'customtkinter'